# Feature Engineering en SQL

A continuación, veremos cómo calcular diferentes variables para el feature engineering utilizando SQL.


In [1]:
import duckdb
import pandas as pd
from sqlalchemy import create_engine

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:

In [2]:
base_path = '/home/fedepicado/'
modelos_path = base_path + 'buckets/b1/modelos/'
db_path = base_path + 'buckets/b1/db/'
dataset_path = base_path + 'buckets/b1/datasets/'
exp_path = base_path + 'buckets/b1/exp/'
dataset_file = 'competencia_02_DQ.parquet'
full_path = dataset_path + dataset_file


In [3]:
%%sql
ROLLBACK;

,Success


In [4]:
%%sql
create or replace table competencia_02_DQ as
select
    *
from read_parquet('../../../buckets/b1/datasets/competencia_02_DQ.parquet')

,Success


In [5]:
df=pd.read_parquet(full_path)

In [6]:
df.shape

(4735593, 149)

In [7]:
campos=df.columns.to_list()
print(campos[-1])
campos.pop(-1) #elimino clase ternaria
print(campos[-1])

clase_ternaria
Visa_mpagominimo


In [9]:
%%sql
CREATE OR REPLACE MACRO suma_sin_null(a, b) AS ifnull(a, 0) + ifnull(b, 0)

,Success


In [10]:
%%sql
CREATE OR REPLACE MACRO div_sin_cero(a, b) as
    case
        when ifnull(b, 0) = 0 THEN NULL
        else ifnull(a, 0) / ifnull(b, 1)
    end

,Success


### Master y Visa, una sola

In [35]:
# Separar las columnas por tipo
master = [col for col in campos if 'Master' in col]
visa = [col for col in campos if 'Visa' in col]

sumas = []

for m_col, v_col in zip(master, visa):
    # Crear la suma usando la función 'suma_sin_null'
    suma = f"\n    suma_sin_null({m_col}, {v_col}) as {m_col.replace('Master_', '')}_total"
    sumas.append(suma)

consulta_sql = f"""
select 
    numero_de_cliente,
    {', '.join(sumas)}
from competencia_01;
"""

print(consulta_sql)


select 
    numero_de_cliente,
    
    suma_sin_null(Master_delinquency, Visa_delinquency) as delinquency_total, 
    suma_sin_null(Master_status, Visa_status) as status_total, 
    suma_sin_null(Master_mfinanciacion_limite, Visa_mfinanciacion_limite) as mfinanciacion_limite_total, 
    suma_sin_null(Master_Fvencimiento, Visa_Fvencimiento) as Fvencimiento_total, 
    suma_sin_null(Master_msaldototal, Visa_msaldototal) as msaldototal_total, 
    suma_sin_null(Master_msaldopesos, Visa_msaldopesos) as msaldopesos_total, 
    suma_sin_null(Master_msaldodolares, Visa_msaldodolares) as msaldodolares_total, 
    suma_sin_null(Master_mconsumospesos, Visa_mconsumospesos) as mconsumospesos_total, 
    suma_sin_null(Master_mconsumosdolares, Visa_mconsumosdolares) as mconsumosdolares_total, 
    suma_sin_null(Master_mlimitecompra, Visa_mlimitecompra) as mlimitecompra_total, 
    suma_sin_null(Master_madelantopesos, Visa_madelantopesos) as madelantopesos_total, 
    suma_sin_null(Master_madelanto

In [11]:
%%sql
create or replace table competencia_02_DQ_agrupacion_var as
select
    *,
    suma_sin_null(Master_delinquency, Visa_delinquency) as delinquency_total,
    suma_sin_null(Master_status, Visa_status) as status_total,
    suma_sin_null(Master_mfinanciacion_limite, Visa_mfinanciacion_limite) as mfinanciacion_limite_total,
    suma_sin_null(Master_msaldototal, Visa_msaldototal) as msaldototal_total,
    suma_sin_null(Master_msaldopesos, Visa_msaldopesos) as msaldopesos_total,
    suma_sin_null(Master_msaldodolares, Visa_msaldodolares) as msaldodolares_total,
    suma_sin_null(Master_mconsumospesos, Visa_mconsumospesos) as mconsumospesos_total,
    suma_sin_null(Master_mconsumosdolares, Visa_mconsumosdolares) as mconsumosdolares_total,
    suma_sin_null(Master_mlimitecompra, Visa_mlimitecompra) as mlimitecompra_total,
    suma_sin_null(Master_madelantopesos, Visa_madelantopesos) as madelantopesos_total,
    suma_sin_null(Master_madelantodolares, Visa_madelantodolares) as madelantodolares_total,
    suma_sin_null(Master_mpagado, Visa_mpagado) as mpagado_total,
    suma_sin_null(Master_mpagospesos, Visa_mpagospesos) as mpagospesos_total,
    suma_sin_null(Master_mpagosdolares, Visa_mpagosdolares) as mpagosdolares_total,
    suma_sin_null(Master_mconsumototal, Visa_mconsumototal) as mconsumototal_total,
    suma_sin_null(Master_cconsumos, Visa_cconsumos) as cconsumos_total,
    suma_sin_null(Master_cadelantosefectivo, Visa_cadelantosefectivo) as cadelantosefectivo_total,
    suma_sin_null(Master_mpagominimo, Visa_mpagominimo) as mpagominimo_total,
    suma_sin_null(ctarjeta_visa, ctarjeta_master) as cant_tarjetas,
    suma_sin_null(ctarjeta_visa_transacciones, ctarjeta_master_transacciones) as cant_transac_tarjetas,
    suma_sin_null(mtarjeta_visa_consumo, mtarjeta_master_consumo) as mtarjetas_consumo,
    suma_sin_null(suma_sin_null(cseguro_vida, cseguro_auto), suma_sin_null(cseguro_vivienda, cseguro_accidentes_personales)) as c_seguros_total
from competencia_02_DQ

,Success


In [22]:
%%sql
select
    foto_mes
    , mfinanciacion_limite_total
from competencia_02_DQ_agrupacion_var
limit 5

,foto_mes,mfinanciacion_limite_total
0,201901,483906.16
1,201902,501305.87
2,201903,549818.98
3,201904,608083.20
4,201905,608083.20


In [13]:
149-46+22

125

In [15]:
columns_to_drop = [
    "Master_delinquency", "Visa_delinquency", "Master_status", "Visa_status",
    "Master_mfinanciacion_limite", "Visa_mfinanciacion_limite", "Master_msaldototal", "Visa_msaldototal",
    "Master_msaldopesos", "Visa_msaldopesos", "Master_msaldodolares", "Visa_msaldodolares",
    "Master_mconsumospesos", "Visa_mconsumospesos", "Master_mconsumosdolares", "Visa_mconsumosdolares",
    "Master_mlimitecompra", "Visa_mlimitecompra", "Master_madelantopesos", "Visa_madelantopesos",
    "Master_madelantodolares", "Visa_madelantodolares", "Master_mpagado", "Visa_mpagado",
    "Master_mpagospesos", "Visa_mpagospesos", "Master_mpagosdolares", "Visa_mpagosdolares",
    "Master_mconsumototal", "Visa_mconsumototal", "Master_cconsumos", "Visa_cconsumos",
    "Master_cadelantosefectivo", "Visa_cadelantosefectivo", "Master_mpagominimo", "Visa_mpagominimo",
    "ctarjeta_visa", "ctarjeta_master", "ctarjeta_visa_transacciones", "ctarjeta_master_transacciones",
    "mtarjeta_visa_consumo", "mtarjeta_master_consumo",
    "cseguro_vida", "cseguro_auto", "cseguro_vivienda", "cseguro_accidentes_personales"
]

In [16]:
len(columns_to_drop)

46

In [17]:
# Ejecutar cada consulta individualmente
for column in columns_to_drop:
    query = f"ALTER TABLE competencia_02_DQ_agrupacion_var DROP COLUMN {column};"
    get_ipython().run_line_magic('sql', query)
    print(f"Columna {column} eliminada.")


Columna Master_delinquency eliminada.
Columna Visa_delinquency eliminada.
Columna Master_status eliminada.
Columna Visa_status eliminada.
Columna Master_mfinanciacion_limite eliminada.
Columna Visa_mfinanciacion_limite eliminada.
Columna Master_msaldototal eliminada.
Columna Visa_msaldototal eliminada.
Columna Master_msaldopesos eliminada.
Columna Visa_msaldopesos eliminada.
Columna Master_msaldodolares eliminada.
Columna Visa_msaldodolares eliminada.
Columna Master_mconsumospesos eliminada.
Columna Visa_mconsumospesos eliminada.
Columna Master_mconsumosdolares eliminada.
Columna Visa_mconsumosdolares eliminada.
Columna Master_mlimitecompra eliminada.
Columna Visa_mlimitecompra eliminada.
Columna Master_madelantopesos eliminada.
Columna Visa_madelantopesos eliminada.
Columna Master_madelantodolares eliminada.
Columna Visa_madelantodolares eliminada.
Columna Master_mpagado eliminada.
Columna Visa_mpagado eliminada.
Columna Master_mpagospesos eliminada.
Columna Visa_mpagospesos eliminada

In [36]:
## guardo el df con la agrupacion de variables
dataset_output= dataset_path + 'competencia_02_DQ_agrupacion_var.parquet'

In [32]:
%%sql
SELECT COUNT(*) AS num_columnas
FROM pragma_table_info('competencia_02_DQ_agrupacion_var');

,num_columnas
0,126


In [33]:
%%sql
ROLLBACK;

,Success


In [37]:
%%sql
COPY competencia_02_DQ_agrupacion_var TO '{{dataset_output}}' (FORMAT PARQUET);

,Success


In [4]:
%%sql
create or replace table competencia_02_DQ as
select
    *
from read_parquet('../../../buckets/b1/datasets/competencia_02_DQ_agrupacion_var.parquet')

,Success


In [5]:
%%sql
select 
    *
from competencia_02_DQ
limit 5


,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,mpagospesos_total,mpagosdolares_total,mconsumototal_total,cconsumos_total,cadelantosefectivo_total,mpagominimo_total,cant_tarjetas,cant_transac_tarjetas,mtarjetas_consumo,c_seguros_total
0,249221109,201901,1,0,1,59,276,7597.550,47433.580,5654.59,...,-54555.28,3.23,33123.58,17.0,0.0,1865.07,2,28,40583.33,2
1,249221109,201902,1,0,1,59,277,3800.930,46605.810,2104.71,...,-40121.86,0.65,28897.85,14.0,0.0,1571.82,2,22,34338.60,2
2,249221109,201903,1,0,1,59,278,4102.670,45348.800,2492.53,...,-34610.56,18.02,37626.45,15.0,0.0,1771.23,2,21,42980.65,2
3,249221109,201904,1,0,1,60,279,4374.990,44872.110,3231.46,...,-43188.68,0.00,115205.33,13.0,0.0,4586.43,2,16,118078.71,2
4,249221109,201905,1,0,1,60,280,3930.055,45187.545,2789.96,...,-117862.69,0.00,213781.65,14.0,0.0,8762.31,2,16,216182.19,2


In [6]:
var_monetarias= ["mrentabilidad","mrentabilidad_annual","mcomisiones","mactivos_margen","mpasivos_margen","mcuenta_corriente",
               "mcaja_ahorro","mcaja_ahorro_adicional","mcaja_ahorro_dolares","mcuentas_saldo","mautoservicio","mtarjeta_visa_consumo",
               "mtarjeta_master_consumo","mtarjeta_master_consumo","mprestamos_prendarios","mprestamos_hipotecarios","mplazo_fijo_dolares",
               "mplazo_fijo_pesos","minversion1_pesos","minversion1_dolares","minversion2","mpayroll","mpayroll2","mcuenta_debitos_automaticos",
               "mttarjeta_visa_debitos_automaticos","mttarjeta_master_debitos_automaticos","mpagomiscuentas","mcajeros_propios_descuentos",
               "mtarjeta_visa_descuentos","mtarjeta_master_descuentos","mcomisiones_mantenimiento","mcomisiones_otras","mforex_buy",
               "mtransferencias_recibidas","mtransferencias_emitidas","mextraccion_autoservicio","mcheques_depositados","mcheques_emitidos",
               "mcheques_depositados_rechazados","mcheques_emitidos_rechazados","matm","matm_other","mfinanciacion_limite_total", 
                "msaldototal_total", "msaldopesos_total","msaldodolares_total", "mconsumospesos_total",
                "mconsumosdolares_total", "mlimitecompra_total", "madelantopesos_total", "madelantodolares_total", 
                "mpagado_total", "mpagospesos_total", "mpagosdolares_total", "mconsumototal_total", 
                "mpagominimo_total", "mtarjetas_consumo"]

## NTILE

## Lag delta r_diff

In [13]:
nuevos_campos = []

for campo in var_monetarias:
    nuevos_campos.append(f", LAG({campo}, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_{campo}")#Lag 1
    nuevos_campos.append(f", LAG({campo}, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_{campo}")#lag 2
    nuevos_campos.append(f", LAG({campo}, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_{campo}")#lag 3
    nuevos_campos.append(f", {campo} - LAG({campo}, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_{campo}")
    nuevos_campos.append(f", ({campo} - LAG({campo}, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) "
                         f"- LAG({campo} - LAG({campo}, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) "
                         f"OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_{campo}")
    nuevos_campos.append(f", AVG({campo}) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS {campo}_media_5")
    nuevos_campos.append(f", LAG(AVG({campo}) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) "
                         f"OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_{campo}_media_5")

# Unimos todas las partes en una sola cadena, eliminando saltos innecesarios
nuevos_campos_str = "\n".join(nuevos_campos)

consulta_sql = f"""
CREATE OR REPLACE TABLE competencia_02_DQ_ft AS
SELECT
    numero_de_cliente,
    foto_mes
    {nuevos_campos_str}
FROM competencia_02_DQ_agrupacion_var;
"""

print(consulta_sql)



CREATE OR REPLACE TABLE competencia_02_DQ_ft AS
SELECT
    numero_de_cliente,
    foto_mes
    , LAG(mrentabilidad, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mrentabilidad
, LAG(mrentabilidad, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mrentabilidad
, LAG(mrentabilidad, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mrentabilidad
, mrentabilidad - LAG(mrentabilidad, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mrentabilidad
, (mrentabilidad - LAG(mrentabilidad, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) - LAG(mrentabilidad - LAG(mrentabilidad, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mrentabilidad
, AVG(mrentabilidad) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mrentabilidad_media_5
, LAG(AVG(mrentabilidad) OVER (PARTITION BY numero_de_

In [37]:
CREATE OR REPLACE TABLE competencia_02_DQ_ft AS
SELECT
    numero_de_cliente,
    foto_mes
    
    , LAG(mrentabilidad, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mrentabilidad
    , LAG(mrentabilidad, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mrentabilidad
    , LAG(mrentabilidad, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mrentabilidad
    , mrentabilidad - LAG(mrentabilidad, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mrentabilidad
    , (mrentabilidad - LAG(mrentabilidad, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(mrentabilidad - LAG(mrentabilidad, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mrentabilidad
    , AVG(mrentabilidad) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mrentabilidad_media_5
    , LAG(AVG(mrentabilidad) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_mrentabilidad_media_5
    
    , LAG(mrentabilidad_annual, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mrentabilidad_annual
    , LAG(mrentabilidad_annual, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mrentabilidad_annual
    , LAG(mrentabilidad_annual, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mrentabilidad_annual
    , mrentabilidad_annual - LAG(mrentabilidad_annual, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mrentabilidad_annual
    , (mrentabilidad_annual - LAG(mrentabilidad_annual, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(mrentabilidad_annual - LAG(mrentabilidad_annual, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mrentabilidad_annual
    , AVG(mrentabilidad_annual) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mrentabilidad_annual_media_5
    , LAG(AVG(mrentabilidad_annual) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_mrentabilidad_annual_media_5
    
    , LAG(mcomisiones, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mcomisiones
    , LAG(mcomisiones, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mcomisiones
    , LAG(mcomisiones, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mcomisiones
    , mcomisiones - LAG(mcomisiones, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mcomisiones
    , (mcomisiones - LAG(mcomisiones, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(mcomisiones - LAG(mcomisiones, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mcomisiones
    , AVG(mcomisiones) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mcomisiones_media_5
    , LAG(AVG(mcomisiones) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_mcomisiones_media_5
    
    , LAG(mactivos_margen, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mactivos_margen
    , LAG(mactivos_margen, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mactivos_margen
    , LAG(mactivos_margen, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mactivos_margen
    , mactivos_margen - LAG(mactivos_margen, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mactivos_margen
    , (mactivos_margen - LAG(mactivos_margen, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(mactivos_margen - LAG(mactivos_margen, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mactivos_margen
    , AVG(mactivos_margen) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mactivos_margen_media_5
    , LAG(AVG(mactivos_margen) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_mactivos_margen_media_5
    
    , LAG(mpasivos_margen, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mpasivos_margen
    , LAG(mpasivos_margen, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mpasivos_margen
    , LAG(mpasivos_margen, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mpasivos_margen
    , mpasivos_margen - LAG(mpasivos_margen, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mpasivos_margen
    , (mpasivos_margen - LAG(mpasivos_margen, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(mpasivos_margen - LAG(mpasivos_margen, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mpasivos_margen
    , AVG(mpasivos_margen) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mpasivos_margen_media_5
    , LAG(AVG(mpasivos_margen) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_mpasivos_margen_media_5
    
    , LAG(mcuenta_corriente, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mcuenta_corriente
    , LAG(mcuenta_corriente, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mcuenta_corriente
    , LAG(mcuenta_corriente, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mcuenta_corriente
    , mcuenta_corriente - LAG(mcuenta_corriente, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mcuenta_corriente
    , (mcuenta_corriente - LAG(mcuenta_corriente, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(mcuenta_corriente - LAG(mcuenta_corriente, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mcuenta_corriente
    , AVG(mcuenta_corriente) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mcuenta_corriente_media_5
    , LAG(AVG(mcuenta_corriente) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_mcuenta_corriente_media_5
    
    , LAG(mcaja_ahorro, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mcaja_ahorro
    , LAG(mcaja_ahorro, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mcaja_ahorro
    , LAG(mcaja_ahorro, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mcaja_ahorro
    , mcaja_ahorro - LAG(mcaja_ahorro, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mcaja_ahorro
    , (mcaja_ahorro - LAG(mcaja_ahorro, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(mcaja_ahorro - LAG(mcaja_ahorro, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mcaja_ahorro
    , AVG(mcaja_ahorro) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mcaja_ahorro_media_5
    , LAG(AVG(mcaja_ahorro) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_mcaja_ahorro_media_5
    
    , LAG(mcaja_ahorro_adicional, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mcaja_ahorro_adicional
    , LAG(mcaja_ahorro_adicional, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mcaja_ahorro_adicional
    , LAG(mcaja_ahorro_adicional, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mcaja_ahorro_adicional
    , mcaja_ahorro_adicional - LAG(mcaja_ahorro_adicional, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mcaja_ahorro_adicional
    , (mcaja_ahorro_adicional - LAG(mcaja_ahorro_adicional, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(mcaja_ahorro_adicional - LAG(mcaja_ahorro_adicional, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mcaja_ahorro_adicional
    , AVG(mcaja_ahorro_adicional) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mcaja_ahorro_adicional_media_5
    , LAG(AVG(mcaja_ahorro_adicional) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_mcaja_ahorro_adicional_media_5
    
    , LAG(mcaja_ahorro_dolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mcaja_ahorro_dolares
    , LAG(mcaja_ahorro_dolares, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mcaja_ahorro_dolares
    , LAG(mcaja_ahorro_dolares, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mcaja_ahorro_dolares
    , mcaja_ahorro_dolares - LAG(mcaja_ahorro_dolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mcaja_ahorro_dolares
    , (mcaja_ahorro_dolares - LAG(mcaja_ahorro_dolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(mcaja_ahorro_dolares - LAG(mcaja_ahorro_dolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mcaja_ahorro_dolares
    , AVG(mcaja_ahorro_dolares) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mcaja_ahorro_dolares_media_5
    , LAG(AVG(mcaja_ahorro_dolares) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_mcaja_ahorro_dolares_media_5
    
    , LAG(mcuentas_saldo, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mcuentas_saldo
    , LAG(mcuentas_saldo, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mcuentas_saldo
    , LAG(mcuentas_saldo, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mcuentas_saldo
    , mcuentas_saldo - LAG(mcuentas_saldo, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mcuentas_saldo
    , (mcuentas_saldo - LAG(mcuentas_saldo, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(mcuentas_saldo - LAG(mcuentas_saldo, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mcuentas_saldo
    , AVG(mcuentas_saldo) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mcuentas_saldo_media_5
    , LAG(AVG(mcuentas_saldo) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_mcuentas_saldo_media_5
    
    , LAG(mautoservicio, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mautoservicio
    , LAG(mautoservicio, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mautoservicio
    , LAG(mautoservicio, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mautoservicio
    , mautoservicio - LAG(mautoservicio, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mautoservicio
    , (mautoservicio - LAG(mautoservicio, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(mautoservicio - LAG(mautoservicio, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mautoservicio
    , AVG(mautoservicio) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mautoservicio_media_5
    , LAG(AVG(mautoservicio) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_mautoservicio_media_5
    
    , LAG(mtarjeta_visa_consumo, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mtarjeta_visa_consumo
    , LAG(mtarjeta_visa_consumo, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mtarjeta_visa_consumo
    , LAG(mtarjeta_visa_consumo, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mtarjeta_visa_consumo
    , mtarjeta_visa_consumo - LAG(mtarjeta_visa_consumo, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mtarjeta_visa_consumo
    , (mtarjeta_visa_consumo - LAG(mtarjeta_visa_consumo, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(mtarjeta_visa_consumo - LAG(mtarjeta_visa_consumo, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mtarjeta_visa_consumo
    , AVG(mtarjeta_visa_consumo) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mtarjeta_visa_consumo_media_5
    , LAG(AVG(mtarjeta_visa_consumo) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_mtarjeta_visa_consumo_media_5
    
    , LAG(mtarjeta_master_consumo, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mtarjeta_master_consumo
    , LAG(mtarjeta_master_consumo, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mtarjeta_master_consumo
    , LAG(mtarjeta_master_consumo, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mtarjeta_master_consumo
    , mtarjeta_master_consumo - LAG(mtarjeta_master_consumo, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mtarjeta_master_consumo
    , (mtarjeta_master_consumo - LAG(mtarjeta_master_consumo, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(mtarjeta_master_consumo - LAG(mtarjeta_master_consumo, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mtarjeta_master_consumo
    , AVG(mtarjeta_master_consumo) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mtarjeta_master_consumo_media_5
    , LAG(AVG(mtarjeta_master_consumo) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_mtarjeta_master_consumo_media_5
    
    , LAG(mtarjeta_master_consumo, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mtarjeta_master_consumo
    , LAG(mtarjeta_master_consumo, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mtarjeta_master_consumo
    , LAG(mtarjeta_master_consumo, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mtarjeta_master_consumo
    , mtarjeta_master_consumo - LAG(mtarjeta_master_consumo, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mtarjeta_master_consumo
    , (mtarjeta_master_consumo - LAG(mtarjeta_master_consumo, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(mtarjeta_master_consumo - LAG(mtarjeta_master_consumo, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mtarjeta_master_consumo
    , AVG(mtarjeta_master_consumo) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mtarjeta_master_consumo_media_5
    , LAG(AVG(mtarjeta_master_consumo) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_mtarjeta_master_consumo_media_5
    
    , LAG(mprestamos_prendarios, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mprestamos_prendarios
    , LAG(mprestamos_prendarios, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mprestamos_prendarios
    , LAG(mprestamos_prendarios, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mprestamos_prendarios
    , mprestamos_prendarios - LAG(mprestamos_prendarios, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mprestamos_prendarios
    , (mprestamos_prendarios - LAG(mprestamos_prendarios, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(mprestamos_prendarios - LAG(mprestamos_prendarios, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mprestamos_prendarios
    , AVG(mprestamos_prendarios) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mprestamos_prendarios_media_5
    , LAG(AVG(mprestamos_prendarios) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_mprestamos_prendarios_media_5
    
    , LAG(mprestamos_hipotecarios, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mprestamos_hipotecarios
    , LAG(mprestamos_hipotecarios, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mprestamos_hipotecarios
    , LAG(mprestamos_hipotecarios, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mprestamos_hipotecarios
    , mprestamos_hipotecarios - LAG(mprestamos_hipotecarios, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mprestamos_hipotecarios
    , (mprestamos_hipotecarios - LAG(mprestamos_hipotecarios, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(mprestamos_hipotecarios - LAG(mprestamos_hipotecarios, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mprestamos_hipotecarios
    , AVG(mprestamos_hipotecarios) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mprestamos_hipotecarios_media_5
    , LAG(AVG(mprestamos_hipotecarios) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_mprestamos_hipotecarios_media_5
    
    , LAG(mplazo_fijo_dolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mplazo_fijo_dolares
    , LAG(mplazo_fijo_dolares, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mplazo_fijo_dolares
    , LAG(mplazo_fijo_dolares, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mplazo_fijo_dolares
    , mplazo_fijo_dolares - LAG(mplazo_fijo_dolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mplazo_fijo_dolares
    , (mplazo_fijo_dolares - LAG(mplazo_fijo_dolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(mplazo_fijo_dolares - LAG(mplazo_fijo_dolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mplazo_fijo_dolares
    , AVG(mplazo_fijo_dolares) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mplazo_fijo_dolares_media_5
    , LAG(AVG(mplazo_fijo_dolares) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_mplazo_fijo_dolares_media_5
    
    , LAG(mplazo_fijo_pesos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mplazo_fijo_pesos
    , LAG(mplazo_fijo_pesos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mplazo_fijo_pesos
    , LAG(mplazo_fijo_pesos, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mplazo_fijo_pesos
    , mplazo_fijo_pesos - LAG(mplazo_fijo_pesos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mplazo_fijo_pesos
    , (mplazo_fijo_pesos - LAG(mplazo_fijo_pesos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(mplazo_fijo_pesos - LAG(mplazo_fijo_pesos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mplazo_fijo_pesos
    , AVG(mplazo_fijo_pesos) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mplazo_fijo_pesos_media_5
    , LAG(AVG(mplazo_fijo_pesos) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_mplazo_fijo_pesos_media_5
    
    , LAG(minversion1_pesos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_minversion1_pesos
    , LAG(minversion1_pesos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_minversion1_pesos
    , LAG(minversion1_pesos, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_minversion1_pesos
    , minversion1_pesos - LAG(minversion1_pesos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_minversion1_pesos
    , (minversion1_pesos - LAG(minversion1_pesos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(minversion1_pesos - LAG(minversion1_pesos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_minversion1_pesos
    , AVG(minversion1_pesos) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS minversion1_pesos_media_5
    , LAG(AVG(minversion1_pesos) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_minversion1_pesos_media_5
    
    , LAG(minversion1_dolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_minversion1_dolares
    , LAG(minversion1_dolares, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_minversion1_dolares
    , LAG(minversion1_dolares, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_minversion1_dolares
    , minversion1_dolares - LAG(minversion1_dolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_minversion1_dolares
    , (minversion1_dolares - LAG(minversion1_dolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(minversion1_dolares - LAG(minversion1_dolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_minversion1_dolares
    , AVG(minversion1_dolares) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS minversion1_dolares_media_5
    , LAG(AVG(minversion1_dolares) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_minversion1_dolares_media_5
    
    , LAG(minversion2, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_minversion2
    , LAG(minversion2, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_minversion2
    , LAG(minversion2, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_minversion2
    , minversion2 - LAG(minversion2, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_minversion2
    , (minversion2 - LAG(minversion2, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(minversion2 - LAG(minversion2, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_minversion2
    , AVG(minversion2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS minversion2_media_5
    , LAG(AVG(minversion2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_minversion2_media_5
    
    , LAG(mpayroll, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mpayroll
    , LAG(mpayroll, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mpayroll
    , LAG(mpayroll, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mpayroll
    , mpayroll - LAG(mpayroll, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mpayroll
    , (mpayroll - LAG(mpayroll, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(mpayroll - LAG(mpayroll, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mpayroll
    , AVG(mpayroll) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mpayroll_media_5
    , LAG(AVG(mpayroll) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_mpayroll_media_5
    
    , LAG(mpayroll2, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mpayroll2
    , LAG(mpayroll2, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mpayroll2
    , LAG(mpayroll2, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mpayroll2
    , mpayroll2 - LAG(mpayroll2, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mpayroll2
    , (mpayroll2 - LAG(mpayroll2, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(mpayroll2 - LAG(mpayroll2, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mpayroll2
    , AVG(mpayroll2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mpayroll2_media_5
    , LAG(AVG(mpayroll2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_mpayroll2_media_5
    
    , LAG(mcuenta_debitos_automaticos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mcuenta_debitos_automaticos
    , LAG(mcuenta_debitos_automaticos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mcuenta_debitos_automaticos
    , LAG(mcuenta_debitos_automaticos, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mcuenta_debitos_automaticos
    , mcuenta_debitos_automaticos - LAG(mcuenta_debitos_automaticos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mcuenta_debitos_automaticos
    , (mcuenta_debitos_automaticos - LAG(mcuenta_debitos_automaticos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(mcuenta_debitos_automaticos - LAG(mcuenta_debitos_automaticos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mcuenta_debitos_automaticos
    , AVG(mcuenta_debitos_automaticos) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mcuenta_debitos_automaticos_media_5
    , LAG(AVG(mcuenta_debitos_automaticos) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_mcuenta_debitos_automaticos_media_5
    
    , LAG(mttarjeta_visa_debitos_automaticos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mttarjeta_visa_debitos_automaticos
    , LAG(mttarjeta_visa_debitos_automaticos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mttarjeta_visa_debitos_automaticos
    , LAG(mttarjeta_visa_debitos_automaticos, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mttarjeta_visa_debitos_automaticos
    , mttarjeta_visa_debitos_automaticos - LAG(mttarjeta_visa_debitos_automaticos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mttarjeta_visa_debitos_automaticos
    , (mttarjeta_visa_debitos_automaticos - LAG(mttarjeta_visa_debitos_automaticos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(mttarjeta_visa_debitos_automaticos - LAG(mttarjeta_visa_debitos_automaticos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mttarjeta_visa_debitos_automaticos
    , AVG(mttarjeta_visa_debitos_automaticos) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mttarjeta_visa_debitos_automaticos_media_5
    , LAG(AVG(mttarjeta_visa_debitos_automaticos) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_mttarjeta_visa_debitos_automaticos_media_5
    
    , LAG(mttarjeta_master_debitos_automaticos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mttarjeta_master_debitos_automaticos
    , LAG(mttarjeta_master_debitos_automaticos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mttarjeta_master_debitos_automaticos
    , LAG(mttarjeta_master_debitos_automaticos, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mttarjeta_master_debitos_automaticos
    , mttarjeta_master_debitos_automaticos - LAG(mttarjeta_master_debitos_automaticos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mttarjeta_master_debitos_automaticos
    , (mttarjeta_master_debitos_automaticos - LAG(mttarjeta_master_debitos_automaticos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(mttarjeta_master_debitos_automaticos - LAG(mttarjeta_master_debitos_automaticos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mttarjeta_master_debitos_automaticos
    , AVG(mttarjeta_master_debitos_automaticos) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mttarjeta_master_debitos_automaticos_media_5
    , LAG(AVG(mttarjeta_master_debitos_automaticos) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_mttarjeta_master_debitos_automaticos_media_5
    
    , LAG(mpagomiscuentas, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mpagomiscuentas
    , LAG(mpagomiscuentas, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mpagomiscuentas
    , LAG(mpagomiscuentas, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mpagomiscuentas
    , mpagomiscuentas - LAG(mpagomiscuentas, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mpagomiscuentas
    , (mpagomiscuentas - LAG(mpagomiscuentas, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(mpagomiscuentas - LAG(mpagomiscuentas, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mpagomiscuentas
    , AVG(mpagomiscuentas) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mpagomiscuentas_media_5
    , LAG(AVG(mpagomiscuentas) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_mpagomiscuentas_media_5
    
    , LAG(mcajeros_propios_descuentos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mcajeros_propios_descuentos
    , LAG(mcajeros_propios_descuentos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mcajeros_propios_descuentos
    , LAG(mcajeros_propios_descuentos, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mcajeros_propios_descuentos
    , mcajeros_propios_descuentos - LAG(mcajeros_propios_descuentos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mcajeros_propios_descuentos
    , (mcajeros_propios_descuentos - LAG(mcajeros_propios_descuentos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(mcajeros_propios_descuentos - LAG(mcajeros_propios_descuentos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mcajeros_propios_descuentos
    , AVG(mcajeros_propios_descuentos) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mcajeros_propios_descuentos_media_5
    , LAG(AVG(mcajeros_propios_descuentos) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_mcajeros_propios_descuentos_media_5
    
    , LAG(mtarjeta_visa_descuentos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mtarjeta_visa_descuentos
    , LAG(mtarjeta_visa_descuentos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mtarjeta_visa_descuentos
    , LAG(mtarjeta_visa_descuentos, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mtarjeta_visa_descuentos
    , mtarjeta_visa_descuentos - LAG(mtarjeta_visa_descuentos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mtarjeta_visa_descuentos
    , (mtarjeta_visa_descuentos - LAG(mtarjeta_visa_descuentos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(mtarjeta_visa_descuentos - LAG(mtarjeta_visa_descuentos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mtarjeta_visa_descuentos
    , AVG(mtarjeta_visa_descuentos) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mtarjeta_visa_descuentos_media_5
    , LAG(AVG(mtarjeta_visa_descuentos) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_mtarjeta_visa_descuentos_media_5
    
    , LAG(mtarjeta_master_descuentos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mtarjeta_master_descuentos
    , LAG(mtarjeta_master_descuentos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mtarjeta_master_descuentos
    , LAG(mtarjeta_master_descuentos, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mtarjeta_master_descuentos
    , mtarjeta_master_descuentos - LAG(mtarjeta_master_descuentos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mtarjeta_master_descuentos
    , (mtarjeta_master_descuentos - LAG(mtarjeta_master_descuentos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(mtarjeta_master_descuentos - LAG(mtarjeta_master_descuentos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mtarjeta_master_descuentos
    , AVG(mtarjeta_master_descuentos) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mtarjeta_master_descuentos_media_5
    , LAG(AVG(mtarjeta_master_descuentos) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_mtarjeta_master_descuentos_media_5
    
    , LAG(mcomisiones_mantenimiento, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mcomisiones_mantenimiento
    , LAG(mcomisiones_mantenimiento, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mcomisiones_mantenimiento
    , LAG(mcomisiones_mantenimiento, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mcomisiones_mantenimiento
    , mcomisiones_mantenimiento - LAG(mcomisiones_mantenimiento, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mcomisiones_mantenimiento
    , (mcomisiones_mantenimiento - LAG(mcomisiones_mantenimiento, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(mcomisiones_mantenimiento - LAG(mcomisiones_mantenimiento, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mcomisiones_mantenimiento
    , AVG(mcomisiones_mantenimiento) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mcomisiones_mantenimiento_media_5
    , LAG(AVG(mcomisiones_mantenimiento) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_mcomisiones_mantenimiento_media_5
    
    , LAG(mcomisiones_otras, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mcomisiones_otras
    , LAG(mcomisiones_otras, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mcomisiones_otras
    , LAG(mcomisiones_otras, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mcomisiones_otras
    , mcomisiones_otras - LAG(mcomisiones_otras, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mcomisiones_otras
    , (mcomisiones_otras - LAG(mcomisiones_otras, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(mcomisiones_otras - LAG(mcomisiones_otras, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mcomisiones_otras
    , AVG(mcomisiones_otras) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mcomisiones_otras_media_5
    , LAG(AVG(mcomisiones_otras) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_mcomisiones_otras_media_5
    
    , LAG(mforex_buy, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mforex_buy
    , LAG(mforex_buy, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mforex_buy
    , LAG(mforex_buy, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mforex_buy
    , mforex_buy - LAG(mforex_buy, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mforex_buy
    , (mforex_buy - LAG(mforex_buy, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(mforex_buy - LAG(mforex_buy, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mforex_buy
    , AVG(mforex_buy) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mforex_buy_media_5
    , LAG(AVG(mforex_buy) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_mforex_buy_media_5
    
    , LAG(mtransferencias_recibidas, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mtransferencias_recibidas
    , LAG(mtransferencias_recibidas, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mtransferencias_recibidas
    , LAG(mtransferencias_recibidas, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mtransferencias_recibidas
    , mtransferencias_recibidas - LAG(mtransferencias_recibidas, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mtransferencias_recibidas
    , (mtransferencias_recibidas - LAG(mtransferencias_recibidas, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(mtransferencias_recibidas - LAG(mtransferencias_recibidas, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mtransferencias_recibidas
    , AVG(mtransferencias_recibidas) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mtransferencias_recibidas_media_5
    , LAG(AVG(mtransferencias_recibidas) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_mtransferencias_recibidas_media_5
    
    , LAG(mtransferencias_emitidas, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mtransferencias_emitidas
    , LAG(mtransferencias_emitidas, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mtransferencias_emitidas
    , LAG(mtransferencias_emitidas, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mtransferencias_emitidas
    , mtransferencias_emitidas - LAG(mtransferencias_emitidas, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mtransferencias_emitidas
    , (mtransferencias_emitidas - LAG(mtransferencias_emitidas, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(mtransferencias_emitidas - LAG(mtransferencias_emitidas, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mtransferencias_emitidas
    , AVG(mtransferencias_emitidas) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mtransferencias_emitidas_media_5
    , LAG(AVG(mtransferencias_emitidas) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_mtransferencias_emitidas_media_5
    
    , LAG(mextraccion_autoservicio, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mextraccion_autoservicio
    , LAG(mextraccion_autoservicio, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mextraccion_autoservicio
    , LAG(mextraccion_autoservicio, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mextraccion_autoservicio
    , mextraccion_autoservicio - LAG(mextraccion_autoservicio, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mextraccion_autoservicio
    , (mextraccion_autoservicio - LAG(mextraccion_autoservicio, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(mextraccion_autoservicio - LAG(mextraccion_autoservicio, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mextraccion_autoservicio
    , AVG(mextraccion_autoservicio) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mextraccion_autoservicio_media_5
    , LAG(AVG(mextraccion_autoservicio) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_mextraccion_autoservicio_media_5
    
    , LAG(mcheques_depositados, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mcheques_depositados
    , LAG(mcheques_depositados, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mcheques_depositados
    , LAG(mcheques_depositados, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mcheques_depositados
    , mcheques_depositados - LAG(mcheques_depositados, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mcheques_depositados
    , (mcheques_depositados - LAG(mcheques_depositados, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(mcheques_depositados - LAG(mcheques_depositados, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mcheques_depositados
    , AVG(mcheques_depositados) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mcheques_depositados_media_5
    , LAG(AVG(mcheques_depositados) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_mcheques_depositados_media_5
    
    , LAG(mcheques_emitidos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mcheques_emitidos
    , LAG(mcheques_emitidos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mcheques_emitidos
    , LAG(mcheques_emitidos, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mcheques_emitidos
    , mcheques_emitidos - LAG(mcheques_emitidos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mcheques_emitidos
    , (mcheques_emitidos - LAG(mcheques_emitidos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(mcheques_emitidos - LAG(mcheques_emitidos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mcheques_emitidos
    , AVG(mcheques_emitidos) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mcheques_emitidos_media_5
    , LAG(AVG(mcheques_emitidos) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_mcheques_emitidos_media_5
    
    , LAG(mcheques_depositados_rechazados, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mcheques_depositados_rechazados
    , LAG(mcheques_depositados_rechazados, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mcheques_depositados_rechazados
    , LAG(mcheques_depositados_rechazados, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mcheques_depositados_rechazados
    , mcheques_depositados_rechazados - LAG(mcheques_depositados_rechazados, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mcheques_depositados_rechazados
    , (mcheques_depositados_rechazados - LAG(mcheques_depositados_rechazados, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(mcheques_depositados_rechazados - LAG(mcheques_depositados_rechazados, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mcheques_depositados_rechazados
    , AVG(mcheques_depositados_rechazados) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mcheques_depositados_rechazados_media_5
    , LAG(AVG(mcheques_depositados_rechazados) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_mcheques_depositados_rechazados_media_5
    
    , LAG(mcheques_emitidos_rechazados, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mcheques_emitidos_rechazados
    , LAG(mcheques_emitidos_rechazados, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mcheques_emitidos_rechazados
    , LAG(mcheques_emitidos_rechazados, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mcheques_emitidos_rechazados
    , mcheques_emitidos_rechazados - LAG(mcheques_emitidos_rechazados, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mcheques_emitidos_rechazados
    , (mcheques_emitidos_rechazados - LAG(mcheques_emitidos_rechazados, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(mcheques_emitidos_rechazados - LAG(mcheques_emitidos_rechazados, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mcheques_emitidos_rechazados
    , AVG(mcheques_emitidos_rechazados) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mcheques_emitidos_rechazados_media_5
    , LAG(AVG(mcheques_emitidos_rechazados) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_mcheques_emitidos_rechazados_media_5
    
    , LAG(matm, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_matm
    , LAG(matm, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_matm
    , LAG(matm, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_matm
    , matm - LAG(matm, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_matm
    , (matm - LAG(matm, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(matm - LAG(matm, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_matm
    , AVG(matm) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS matm_media_5
    , LAG(AVG(matm) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_matm_media_5
    
    , LAG(matm_other, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_matm_other
    , LAG(matm_other, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_matm_other
    , LAG(matm_other, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_matm_other
    , matm_other - LAG(matm_other, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_matm_other
    , (matm_other - LAG(matm_other, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(matm_other - LAG(matm_other, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_matm_other
    , AVG(matm_other) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS matm_other_media_5
    , LAG(AVG(matm_other) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_matm_other_media_5
    
    , LAG(mfinanciacion_limite_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mfinanciacion_limite_total
    , LAG(mfinanciacion_limite_total, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mfinanciacion_limite_total
    , LAG(mfinanciacion_limite_total, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mfinanciacion_limite_total
    , mfinanciacion_limite_total - LAG(mfinanciacion_limite_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mfinanciacion_limite_total
    , (mfinanciacion_limite_total - LAG(mfinanciacion_limite_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(mfinanciacion_limite_total - LAG(mfinanciacion_limite_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mfinanciacion_limite_total
    , AVG(mfinanciacion_limite_total) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mfinanciacion_limite_total_media_5
    , LAG(AVG(mfinanciacion_limite_total) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_mfinanciacion_limite_total_media_5
    
    , LAG(msaldototal_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_msaldototal_total
    , LAG(msaldototal_total, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_msaldototal_total
    , LAG(msaldototal_total, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_msaldototal_total
    , msaldototal_total - LAG(msaldototal_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_msaldototal_total
    , (msaldototal_total - LAG(msaldototal_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(msaldototal_total - LAG(msaldototal_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_msaldototal_total
    , AVG(msaldototal_total) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS msaldototal_total_media_5
    , LAG(AVG(msaldototal_total) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_msaldototal_total_media_5
    
    , LAG(msaldopesos_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_msaldopesos_total
    , LAG(msaldopesos_total, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_msaldopesos_total
    , LAG(msaldopesos_total, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_msaldopesos_total
    , msaldopesos_total - LAG(msaldopesos_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_msaldopesos_total
    , (msaldopesos_total - LAG(msaldopesos_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(msaldopesos_total - LAG(msaldopesos_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_msaldopesos_total
    , AVG(msaldopesos_total) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS msaldopesos_total_media_5
    , LAG(AVG(msaldopesos_total) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_msaldopesos_total_media_5
    
    , LAG(msaldodolares_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_msaldodolares_total
    , LAG(msaldodolares_total, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_msaldodolares_total
    , LAG(msaldodolares_total, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_msaldodolares_total
    , msaldodolares_total - LAG(msaldodolares_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_msaldodolares_total
    , (msaldodolares_total - LAG(msaldodolares_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(msaldodolares_total - LAG(msaldodolares_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_msaldodolares_total
    , AVG(msaldodolares_total) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS msaldodolares_total_media_5
    , LAG(AVG(msaldodolares_total) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_msaldodolares_total_media_5
    
    , LAG(mconsumospesos_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mconsumospesos_total
    , LAG(mconsumospesos_total, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mconsumospesos_total
    , LAG(mconsumospesos_total, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mconsumospesos_total
    , mconsumospesos_total - LAG(mconsumospesos_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mconsumospesos_total
    , (mconsumospesos_total - LAG(mconsumospesos_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(mconsumospesos_total - LAG(mconsumospesos_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mconsumospesos_total
    , AVG(mconsumospesos_total) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mconsumospesos_total_media_5
    , LAG(AVG(mconsumospesos_total) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_mconsumospesos_total_media_5
    
    , LAG(mconsumosdolares_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mconsumosdolares_total
    , LAG(mconsumosdolares_total, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mconsumosdolares_total
    , LAG(mconsumosdolares_total, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mconsumosdolares_total
    , mconsumosdolares_total - LAG(mconsumosdolares_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mconsumosdolares_total
    , (mconsumosdolares_total - LAG(mconsumosdolares_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(mconsumosdolares_total - LAG(mconsumosdolares_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mconsumosdolares_total
    , AVG(mconsumosdolares_total) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mconsumosdolares_total_media_5
    , LAG(AVG(mconsumosdolares_total) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_mconsumosdolares_total_media_5
    
    , LAG(mlimitecompra_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mlimitecompra_total
    , LAG(mlimitecompra_total, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mlimitecompra_total
    , LAG(mlimitecompra_total, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mlimitecompra_total
    , mlimitecompra_total - LAG(mlimitecompra_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mlimitecompra_total
    , (mlimitecompra_total - LAG(mlimitecompra_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(mlimitecompra_total - LAG(mlimitecompra_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mlimitecompra_total
    , AVG(mlimitecompra_total) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mlimitecompra_total_media_5
    , LAG(AVG(mlimitecompra_total) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_mlimitecompra_total_media_5
    
    , LAG(madelantopesos_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_madelantopesos_total
    , LAG(madelantopesos_total, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_madelantopesos_total
    , LAG(madelantopesos_total, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_madelantopesos_total
    , madelantopesos_total - LAG(madelantopesos_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_madelantopesos_total
    , (madelantopesos_total - LAG(madelantopesos_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(madelantopesos_total - LAG(madelantopesos_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_madelantopesos_total
    , AVG(madelantopesos_total) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS madelantopesos_total_media_5
    , LAG(AVG(madelantopesos_total) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_madelantopesos_total_media_5
    
    , LAG(madelantodolares_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_madelantodolares_total
    , LAG(madelantodolares_total, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_madelantodolares_total
    , LAG(madelantodolares_total, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_madelantodolares_total
    , madelantodolares_total - LAG(madelantodolares_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_madelantodolares_total
    , (madelantodolares_total - LAG(madelantodolares_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(madelantodolares_total - LAG(madelantodolares_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_madelantodolares_total
    , AVG(madelantodolares_total) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS madelantodolares_total_media_5
    , LAG(AVG(madelantodolares_total) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_madelantodolares_total_media_5
    
    , LAG(mpagado_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mpagado_total
    , LAG(mpagado_total, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mpagado_total
    , LAG(mpagado_total, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mpagado_total
    , mpagado_total - LAG(mpagado_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mpagado_total
    , (mpagado_total - LAG(mpagado_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(mpagado_total - LAG(mpagado_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mpagado_total
    , AVG(mpagado_total) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mpagado_total_media_5
    , LAG(AVG(mpagado_total) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_mpagado_total_media_5
    
    , LAG(mpagospesos_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mpagospesos_total
    , LAG(mpagospesos_total, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mpagospesos_total
    , LAG(mpagospesos_total, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mpagospesos_total
    , mpagospesos_total - LAG(mpagospesos_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mpagospesos_total
    , (mpagospesos_total - LAG(mpagospesos_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(mpagospesos_total - LAG(mpagospesos_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mpagospesos_total
    , AVG(mpagospesos_total) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mpagospesos_total_media_5
    , LAG(AVG(mpagospesos_total) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_mpagospesos_total_media_5
    
    , LAG(mpagosdolares_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mpagosdolares_total
    , LAG(mpagosdolares_total, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mpagosdolares_total
    , LAG(mpagosdolares_total, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mpagosdolares_total
    , mpagosdolares_total - LAG(mpagosdolares_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mpagosdolares_total
    , (mpagosdolares_total - LAG(mpagosdolares_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(mpagosdolares_total - LAG(mpagosdolares_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mpagosdolares_total
    , AVG(mpagosdolares_total) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mpagosdolares_total_media_5
    , LAG(AVG(mpagosdolares_total) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_mpagosdolares_total_media_5
    
    , LAG(mconsumototal_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mconsumototal_total
    , LAG(mconsumototal_total, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mconsumototal_total
    , LAG(mconsumototal_total, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mconsumototal_total
    , mconsumototal_total - LAG(mconsumototal_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mconsumototal_total
    , (mconsumototal_total - LAG(mconsumototal_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(mconsumototal_total - LAG(mconsumototal_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mconsumototal_total
    , AVG(mconsumototal_total) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mconsumototal_total_media_5
    , LAG(AVG(mconsumototal_total) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_mconsumototal_total_media_5
    
    , LAG(mpagominimo_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mpagominimo_total
    , LAG(mpagominimo_total, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mpagominimo_total
    , LAG(mpagominimo_total, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mpagominimo_total
    , mpagominimo_total - LAG(mpagominimo_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mpagominimo_total
    , (mpagominimo_total - LAG(mpagominimo_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(mpagominimo_total - LAG(mpagominimo_total, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mpagominimo_total
    , AVG(mpagominimo_total) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mpagominimo_total_media_5
    , LAG(AVG(mpagominimo_total) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_mpagominimo_total_media_5
    
    , LAG(mtarjetas_consumo, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mtarjetas_consumo
    , LAG(mtarjetas_consumo, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mtarjetas_consumo
    , LAG(mtarjetas_consumo, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mtarjetas_consumo
    , mtarjetas_consumo - LAG(mtarjetas_consumo, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif1_mtarjetas_consumo
    , (mtarjetas_consumo - LAG(mtarjetas_consumo, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)) 
      - LAG(mtarjetas_consumo - LAG(mtarjetas_consumo, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS dif2_mtarjetas_consumo
    , AVG(mtarjetas_consumo) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS mtarjetas_consumo_media_5
    , LAG(AVG(mtarjetas_consumo) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes ROWS BETWEEN 4 PRECEDING AND CURRENT ROW), 1) 
      OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lagged_mtarjetas_consumo_media_5
    
FROM competencia_01;

,Success


In [36]:
# %%sql
# select *
# from competencia_01_lag_delta_ratios
# where foto_mes =202104 or foto_mes =202106


,numero_de_cliente,foto_mes,lag_1_ctrx_quarter,delta_1_ctrx_quarter,lag_2_ctrx_quarter,delta_2_ctrx_quarter,ratio_ctrx_quarter,lag_1_cpayroll_trx,delta_1_cpayroll_trx,lag_2_cpayroll_trx,...,lag_1_cprestamos_personales,delta_1_cprestamos_personales,lag_2_cprestamos_personales,delta_2_cprestamos_personales,ratio_cprestamos_personales,lag_1_mcuenta_corriente,delta_1_mcuenta_corriente,lag_2_mcuenta_corriente,delta_2_mcuenta_corriente,ratio_mcuenta_corriente
0,249221109,202104,174.0,14.0,161.0,27.0,0.518519,2.0,-1.0,1.0,...,0.0,0.0,0.0,0.0,NaN,-485.88,485.88,0.00,0.00,NaN
1,249278864,202104,110.0,0.0,NaN,NaN,NaN,1.0,0.0,NaN,...,0.0,0.0,NaN,NaN,NaN,0.00,0.00,NaN,NaN,NaN
2,249296729,202104,63.0,8.0,55.0,16.0,0.500000,2.0,-1.0,0.0,...,0.0,0.0,0.0,0.0,NaN,2990.46,-2990.46,3196.71,-3196.71,0.935481
3,249470734,202104,116.0,-4.0,112.0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,0.00,0.00,0.00,0.00,NaN
4,249590846,202104,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,0.00,0.00,0.00,0.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328961,1588492803,202104,9.0,39.0,NaN,NaN,NaN,0.0,0.0,NaN,...,0.0,0.0,NaN,NaN,NaN,0.00,0.00,NaN,NaN,NaN
328962,1588725801,202104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
328963,1588924016,202104,2.0,8.0,NaN,NaN,NaN,0.0,2.0,NaN,...,0.0,0.0,NaN,NaN,NaN,0.00,-1416.04,NaN,NaN,NaN
328964,1591753879,202104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Ahora voy con el promedio de cada campo, haciendo dev

In [38]:
nuevos_features = ""
for campo in campos:
    # Calcular el promedio móvil de los últimos 3 meses
    nuevos_features += f"\n    , avg({campo}) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_{campo}"
    
    # Calcular la desviación del valor actual con respecto al promedio móvil
    nuevos_features += f"\n    , {campo} - avg_3_{campo} as dev_3_{campo}"

# Construir la consulta SQL
consulta_sql = f"""
CREATE OR REPLACE TABLE competencia_01_DQ_avg AS
select
    numero_de_cliente
    , foto_mes 
    {nuevos_features.strip()}
FROM competencia_01_DQ
where foto_mes =202104 or foto_mes =202106
"""

print(consulta_sql)


CREATE OR REPLACE TABLE competencia_01_DQ_avg AS
select
    numero_de_cliente
    , foto_mes 
    , avg(mpayroll) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mpayroll
    , mpayroll - avg_3_mpayroll as dev_3_mpayroll
    , avg(ctrx_quarter) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_ctrx_quarter
    , ctrx_quarter - avg_3_ctrx_quarter as dev_3_ctrx_quarter
    , avg(mprestamos_personales) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mprestamos_personales
    , mprestamos_personales - avg_3_mprestamos_personales as dev_3_mprestamos_personales
    , avg(mactivos_margen) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mactivos_margen
    , mactivos_margen - avg_3_mactivos_margen as dev_3_mactivos_margen
    , avg(mpasivos_margen) over (partition by

In [39]:
%%sql

CREATE OR REPLACE TABLE competencia_01_DQ_avg AS
select
    numero_de_cliente
    , foto_mes 
    , avg(mpayroll) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mpayroll
    , mpayroll - avg_3_mpayroll as dev_3_mpayroll
    , avg(ctrx_quarter) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_ctrx_quarter
    , ctrx_quarter - avg_3_ctrx_quarter as dev_3_ctrx_quarter
    , avg(mprestamos_personales) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mprestamos_personales
    , mprestamos_personales - avg_3_mprestamos_personales as dev_3_mprestamos_personales
    , avg(mactivos_margen) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mactivos_margen
    , mactivos_margen - avg_3_mactivos_margen as dev_3_mactivos_margen
    , avg(mpasivos_margen) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mpasivos_margen
    , mpasivos_margen - avg_3_mpasivos_margen as dev_3_mpasivos_margen
    , avg(mcuentas_saldo) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mcuentas_saldo
    , mcuentas_saldo - avg_3_mcuentas_saldo as dev_3_mcuentas_saldo
    , avg(mcaja_ahorro) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mcaja_ahorro
    , mcaja_ahorro - avg_3_mcaja_ahorro as dev_3_mcaja_ahorro
    , avg(internet) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_internet
    , internet - avg_3_internet as dev_3_internet
    , avg(tcallcenter) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_tcallcenter
    , tcallcenter - avg_3_tcallcenter as dev_3_tcallcenter
    , avg(mpagomiscuentas) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mpagomiscuentas
    , mpagomiscuentas - avg_3_mpagomiscuentas as dev_3_mpagomiscuentas
    , avg(ccaja_ahorro) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_ccaja_ahorro
    , ccaja_ahorro - avg_3_ccaja_ahorro as dev_3_ccaja_ahorro
    , avg(mcomisiones_mantenimiento) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mcomisiones_mantenimiento
    , mcomisiones_mantenimiento - avg_3_mcomisiones_mantenimiento as dev_3_mcomisiones_mantenimiento
    , avg(ccomisiones_mantenimiento) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_ccomisiones_mantenimiento
    , ccomisiones_mantenimiento - avg_3_ccomisiones_mantenimiento as dev_3_ccomisiones_mantenimiento
FROM competencia_01_DQ
where foto_mes =202104 or foto_mes =202106




,Success


## reg_slope

In [41]:
nuevos_features = ""
for campo in campos:
  nuevos_features +=  f"\n    ,regr_slope({campo}, cliente_antiguedad) over ventana_3 as {campo}_slope_3"


consulta_sql = f"""
CREATE OR REPLACE TABLE competencia_01_regr_slope AS
select
    numero_de_cliente
    , foto_mes 
    {nuevos_features.strip()}
from competencia_01
window ventana_3 as (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row)
"""

print(consulta_sql)


CREATE OR REPLACE TABLE competencia_01_regr_slope AS
select
    numero_de_cliente
    , foto_mes 
    ,regr_slope(mpayroll, cliente_antiguedad) over ventana_3 as mpayroll_slope_3
    ,regr_slope(ctrx_quarter, cliente_antiguedad) over ventana_3 as ctrx_quarter_slope_3
    ,regr_slope(mprestamos_personales, cliente_antiguedad) over ventana_3 as mprestamos_personales_slope_3
    ,regr_slope(mactivos_margen, cliente_antiguedad) over ventana_3 as mactivos_margen_slope_3
    ,regr_slope(mpasivos_margen, cliente_antiguedad) over ventana_3 as mpasivos_margen_slope_3
    ,regr_slope(mcuentas_saldo, cliente_antiguedad) over ventana_3 as mcuentas_saldo_slope_3
    ,regr_slope(mcaja_ahorro, cliente_antiguedad) over ventana_3 as mcaja_ahorro_slope_3
    ,regr_slope(internet, cliente_antiguedad) over ventana_3 as internet_slope_3
    ,regr_slope(tcallcenter, cliente_antiguedad) over ventana_3 as tcallcenter_slope_3
    ,regr_slope(mpagomiscuentas, cliente_antiguedad) over ventana_3 as mpagomiscuent

In [44]:
%%sql
CREATE OR REPLACE TABLE competencia_01_DQ_regr_slope AS
select
    numero_de_cliente
    ,foto_mes 
    ,regr_slope(mpayroll, cliente_antiguedad) over ventana_3 as mpayroll_slope_3
    ,regr_slope(ctrx_quarter, cliente_antiguedad) over ventana_3 as ctrx_quarter_slope_3
    ,regr_slope(mprestamos_personales, cliente_antiguedad) over ventana_3 as mprestamos_personales_slope_3
    ,regr_slope(mactivos_margen, cliente_antiguedad) over ventana_3 as mactivos_margen_slope_3
    ,regr_slope(mpasivos_margen, cliente_antiguedad) over ventana_3 as mpasivos_margen_slope_3
    ,regr_slope(mcuentas_saldo, cliente_antiguedad) over ventana_3 as mcuentas_saldo_slope_3
    ,regr_slope(mcaja_ahorro, cliente_antiguedad) over ventana_3 as mcaja_ahorro_slope_3
    ,regr_slope(internet, cliente_antiguedad) over ventana_3 as internet_slope_3
    ,regr_slope(tcallcenter, cliente_antiguedad) over ventana_3 as tcallcenter_slope_3
    ,regr_slope(mpagomiscuentas, cliente_antiguedad) over ventana_3 as mpagomiscuentas_slope_3
    ,regr_slope(ccaja_ahorro, cliente_antiguedad) over ventana_3 as ccaja_ahorro_slope_3
    ,regr_slope(mcomisiones_mantenimiento, cliente_antiguedad) over ventana_3 as mcomisiones_mantenimiento_slope_3
    ,regr_slope(ccomisiones_mantenimiento, cliente_antiguedad) over ventana_3 as ccomisiones_mantenimiento_slope_3
from competencia_01_DQ
window ventana_3 as (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row)


,Success


In [45]:
%%sql
select 
    *
from competencia_01_DQ_regr_slope

,numero_de_cliente,foto_mes,mpayroll_slope_3,ctrx_quarter_slope_3,mprestamos_personales_slope_3,mactivos_margen_slope_3,mpasivos_margen_slope_3,mcuentas_saldo_slope_3,mcaja_ahorro_slope_3,internet_slope_3,tcallcenter_slope_3,mpagomiscuentas_slope_3,ccaja_ahorro_slope_3,mcomisiones_mantenimiento_slope_3,ccomisiones_mantenimiento_slope_3
0,249223005,202101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,249223005,202102,16316.430,26.0,0.0,958.030,-160.780,10385.810,-5887.650,0.0,0.0,0.0,0.0,0.0,0.0
2,249223005,202103,5164.135,9.5,0.0,315.990,-62.735,-2649.690,-2105.080,0.0,0.5,0.0,0.0,0.0,0.0
3,249223005,202104,8159.975,2.9,0.0,202.097,-86.253,815.881,-106.823,0.0,0.1,0.0,0.0,0.0,0.0
4,249223005,202105,6988.969,-11.8,0.0,-57.591,84.583,-3422.456,8390.396,0.0,-0.1,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
981941,1541018080,202104,-443.596,-21.4,0.0,-76.472,-1032.619,22200.516,4729.535,0.0,0.0,0.0,0.0,0.0,0.0
981942,1541018080,202105,658.243,-16.9,0.0,296.678,-501.846,15068.193,4163.844,0.0,0.0,0.0,0.0,0.0,0.0
981943,1541018080,202106,24321.358,4.7,0.0,516.806,400.665,36037.387,28275.512,0.0,0.0,0.0,0.0,0.0,0.0
981944,1542134813,202101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
nuevos_features = ""
for campo in campos:
  nuevos_features +=  f"\n    ,percent_rank() OVER (ORDER BY {campo}) as percent_rank_{campo}"


consulta_sql = f"""
CREATE OR REPLACE TABLE competencia_01_percent_rank AS
select
    numero_de_cliente
    , foto_mes 
    {nuevos_features.strip()}
from competencia_01

"""

print(consulta_sql)


CREATE OR REPLACE TABLE competencia_01_percent_rank AS
select
    numero_de_cliente
    , foto_mes 
    ,percent_rank() OVER (ORDER BY mpayroll) as percent_rank_mpayroll
    ,percent_rank() OVER (ORDER BY ctrx_quarter) as percent_rank_ctrx_quarter
    ,percent_rank() OVER (ORDER BY mprestamos_personales) as percent_rank_mprestamos_personales
    ,percent_rank() OVER (ORDER BY mactivos_margen) as percent_rank_mactivos_margen
    ,percent_rank() OVER (ORDER BY mpasivos_margen) as percent_rank_mpasivos_margen
    ,percent_rank() OVER (ORDER BY mcuentas_saldo) as percent_rank_mcuentas_saldo
    ,percent_rank() OVER (ORDER BY mcaja_ahorro) as percent_rank_mcaja_ahorro
    ,percent_rank() OVER (ORDER BY internet) as percent_rank_internet
    ,percent_rank() OVER (ORDER BY tcallcenter) as percent_rank_tcallcenter
    ,percent_rank() OVER (ORDER BY mpagomiscuentas) as percent_rank_mpagomiscuentas
    ,percent_rank() OVER (ORDER BY ccaja_ahorro) as percent_rank_ccaja_ahorro
    ,percent_rank()

In [47]:
%%sql

CREATE OR REPLACE TABLE competencia_01_DQ_percent_rank AS
select
    numero_de_cliente
    , foto_mes 
    ,percent_rank() OVER (ORDER BY mpayroll) as percent_rank_mpayroll
    ,percent_rank() OVER (ORDER BY ctrx_quarter) as percent_rank_ctrx_quarter
    ,percent_rank() OVER (ORDER BY mprestamos_personales) as percent_rank_mprestamos_personales
    ,percent_rank() OVER (ORDER BY mactivos_margen) as percent_rank_mactivos_margen
    ,percent_rank() OVER (ORDER BY mpasivos_margen) as percent_rank_mpasivos_margen
    ,percent_rank() OVER (ORDER BY mcuentas_saldo) as percent_rank_mcuentas_saldo
    ,percent_rank() OVER (ORDER BY mcaja_ahorro) as percent_rank_mcaja_ahorro
    ,percent_rank() OVER (ORDER BY internet) as percent_rank_internet
    ,percent_rank() OVER (ORDER BY tcallcenter) as percent_rank_tcallcenter
    ,percent_rank() OVER (ORDER BY mpagomiscuentas) as percent_rank_mpagomiscuentas
    ,percent_rank() OVER (ORDER BY ccaja_ahorro) as percent_rank_ccaja_ahorro
    ,percent_rank() OVER (ORDER BY mcomisiones_mantenimiento) as percent_rank_mcomisiones_mantenimiento
    ,percent_rank() OVER (ORDER BY ccomisiones_mantenimiento) as percent_rank_ccomisiones_mantenimiento
from competencia_01_DQ
WHERE foto_mes = 202104 OR foto_mes = 202106

,Success


In [49]:
%%sql 
create or replace table competencia_01_DQ_fe as
SELECT * 
FROM (
    SELECT * 
    FROM competencia_01_DQ
    WHERE foto_mes = 202104 OR foto_mes = 202106
) AS base

LEFT JOIN competencia_01_NTILE_10 as ntile_10
ON base.numero_de_cliente = ntile_10.numero_de_cliente 
    AND base.foto_mes = ntile_10.foto_mes  

LEFT JOIN competencia_01_NTILE_20 as ntile_20
ON ntile_10.numero_de_cliente = ntile_20.numero_de_cliente 
    AND ntile_10.foto_mes = ntile_20.foto_mes  

LEFT JOIN competencia_01_NTILE_30 as ntile_30
ON ntile_20.numero_de_cliente = ntile_30.numero_de_cliente 
    AND ntile_20.foto_mes = ntile_30.foto_mes  

LEFT JOIN competencia_01_DQ_avg as avg 
ON ntile_30.numero_de_cliente = avg.numero_de_cliente 
    AND ntile_30.foto_mes = avg.foto_mes  
    
LEFT JOIN competencia_01_DQ_regr_slope as reg 
ON avg.numero_de_cliente = reg.numero_de_cliente 
    AND avg.foto_mes = reg.foto_mes

LEFT JOIN competencia_01_DQ_percent_rank as p_rank
ON reg.numero_de_cliente = p_rank.numero_de_cliente 
    AND reg.foto_mes = p_rank.foto_mes

,Success


In [50]:
%%sql
COPY competencia_01_DQ_fe TO '../../../datasets/competencia_01_DQ_fe_2.csv' (FORMAT CSV, HEADER TRUE)

,Success


In [52]:
%%sql
select 
    *
from competencia_01_DQ_fe
limit 4

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,percent_rank_mactivos_margen,percent_rank_mpasivos_margen,percent_rank_mcuentas_saldo,percent_rank_mcaja_ahorro,percent_rank_internet,percent_rank_tcallcenter,percent_rank_mpagomiscuentas,percent_rank_ccaja_ahorro,percent_rank_mcomisiones_mantenimiento,percent_rank_ccomisiones_mantenimiento
0,345821831,202104,1,0,0,50,259,3164.74,143431.55,359.89,...,0.849902,0.805329,0.607411,0.842248,0.0,0.0,0.0,0.0,0.713486,0.669144
1,345821831,202106,1,0,0,50,261,4650.94,130432.35,2305.66,...,0.817215,0.783053,0.444047,0.813615,0.0,0.0,0.0,0.0,0.856909,0.669144
2,345847792,202104,1,0,0,54,89,3762.77,35013.28,4724.67,...,0.287359,0.562184,0.017680,0.000088,0.0,0.0,0.0,0.0,0.960418,0.669144
3,345847792,202106,1,0,0,54,91,3541.68,40052.31,4201.19,...,0.306093,0.605770,0.017558,0.000088,0.0,0.0,0.0,0.0,0.960418,0.669144
